In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tp
import tensorflow.keras.layers as layers
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
import glob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# !unzip "/content/drive/MyDrive/Projects/Github/MNIST DATA/Train_UQcUa52.zip"

In [28]:
# Read CSV
data = pd.read_csv('/content/train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/Projects/Github/MNIST DATA/Test_fCbTej3_0j1gHmj.csv')

In [ ]:
data.head()

,filename,label
0,0.png,4
1,1.png,9
2,2.png,1
3,3.png,7
4,4.png,3


In [31]:
Y = np.array(data['label'])

In [30]:
X=[]
for i in data['filename']:
    path = '/content/Images/train/'+str(i)
    img=cv2.imread(path)
    img = cv2.resize(img,(32,32))
    X.append(img)
    
X = np.array(X)
X = X / 255

In [32]:
Y=tf.one_hot(Y,10)

In [15]:
Y = np.array(Y)

In [16]:
Y[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [20]:
X_normalized = X.astype('float32') / 255

In [ ]:
# X_train,X_test,y_train,y_test=train_test_split(X_normalized,Y,random_state=42)

In [34]:
base_model = VGG16(include_top= False, weights = 'imagenet', input_shape = X[0].shape)

In [35]:
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [36]:
model = tf.keras.Sequential([
    # layers.Conv2D(32,(3,3),input_shape=(28,28,3),activation='relu'),
    # layers.MaxPooling2D(2,2),
    # layers.Conv2D(16,(3,3),activation='relu'),
    # layers.MaxPooling2D(2,2),
    
    # layers.Flatten(),
    # layers.Dense(100,activation='relu'),
    # layers.Dropout(0.2),
    # layers.Dense(50,activation='relu'),
    # layers.Dropout(0.2),
    # layers.Dense(50,activation='relu'),
    # layers.Dense(10,activation='softmax')
    base_model,
    layers.Flatten(),
    layers.Dense(50,activation = 'relu'),
    layers.Dense(20,activation = 'relu'),
    layers.Dense(10,activation = 'softmax'),

    
    
])

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                25650     
_________________________________________________________________
dense_4 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                210       
Total params: 14,741,568
Trainable params: 26,880
Non-trainable params: 14,714,688
_________________________________________________________________


In [38]:
model.compile(optimizer=tf.keras.optimizers.Adam(3e-4),metrics=['accuracy'],loss=tf.keras.losses.CategoricalCrossentropy())

In [39]:
model.fit(X_normalized,Y,epochs=5, validation_split = 0.2, batch_size = 28)

Epoch 1/5
1400/1400 [==============================] - 466s 332ms/step - loss: 0.9030 - accuracy: 0.7514 - val_loss: 0.4279 - val_accuracy: 0.8828
Epoch 2/5
1400/1400 [==============================] - 465s 332ms/step - loss: 0.3293 - accuracy: 0.9059 - val_loss: 0.2785 - val_accuracy: 0.9203
Epoch 3/5
1400/1400 [==============================] - 464s 332ms/step - loss: 0.2407 - accuracy: 0.9295 - val_loss: 0.2277 - val_accuracy: 0.9331
Epoch 4/5
1400/1400 [==============================] - 465s 332ms/step - loss: 0.2002 - accuracy: 0.9410 - val_loss: 0.1971 - val_accuracy: 0.9407
Epoch 5/5
1400/1400 [==============================] - 464s 332ms/step - loss: 0.1748 - accuracy: 0.9470 - val_loss: 0.1820 - val_accuracy: 0.9451


In [ ]:
model.evaluate(X_test,y_test)

383/383 [==============================] - 4s 10ms/step - loss: 0.0416 - accuracy: 0.9906


[0.04157872125506401, 0.990612268447876]

In [40]:
model.save('Digit_Identifier_VGG16.model',save_format='h5')

In [42]:
!cp /content/Digit_Identifier_VGG16.model "/content/drive/MyDrive/Projects/Github/MNIST DATA"